In [1]:
import pandas

# Считайте таблицу с признаками из файла features.csv
features = pandas.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


In [2]:
match_result_columns = [
    'duration',
    'radiant_win',
    'tower_status_radiant',
    'tower_status_dire',
    'barracks_status_radiant',
    'barracks_status_dire'
]

# Удалите признаки, связанные с итогами матча
match_result_data = features[match_result_columns]
y = match_result_data['radiant_win']

match_progress_data = features.drop(match_result_columns, axis=1)

In [19]:
progress_count = match_progress_data.count()

In [23]:
partial_columns_info = progress_count[progress_count != progress_count.max()]
print(partial_columns_info)

print(u"\nВопрос: Много ли пропусков в данных?")
print(u"Ответ: из {} признаков, частично заполнены {}".format(len(progress_count), len(partial_columns_info)))

print(u"\nВопрос: Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать" 
      u"обоснование, почему их значения могут быть пропущены.")
print(u"Ответ: Список названий признаков, имеющих пропуски:")
print(", ".join([key for key in partial_columns_info.keys()]))

print(u"""
Основание пропусков для признаков:
* first_blood_time: игровое время первой крови
* first_blood_team: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
* first_blood_player1: игрок, причастный к событию
* first_blood_player2: второй игрок, причастный к событию

Понятно из описания задачи: "Если событие "первая кровь" не успело произойти за первые 5 минут, 
то признаки принимают пропущенное значение"

Признаки:
* radiant_bottle_time
* radiant_courier_time
* radiant_flying_courier_time
* radiant_first_ward_time
* dire_bottle_time
* dire_courier_time
* dire_flying_courier_time
* dire_first_ward_time

описывают время наступления некоторого необязательного события игры. Судя по всему - в некоторых матчах данное
событие не наступило в первые 5 минут матча.
""")

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

Вопрос: Много ли пропусков в данных?
Ответ: из 102 признаков, частично заполнены 12

Вопрос: Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них датьобоснование, почему их значения могут быть пропущены.
Ответ: Список названий признаков, имеющих пропуски:
first_blood_time, first_blood_team, first_blood_player1, first_blood_player2, radiant_bottle_time, radiant_courier_time, radiant_flying_courier_time, radiant_first_ward_time, dire_bottle_time, dire_courier_time, dire_flying_courier_time, dire_first_ward_tim

In [3]:
# Замените пропуски на нули с помощью функции fillna()
X = match_progress_data.fillna(0)

In [27]:
print(u"Вопрос: Какой столбец содержит целевую переменную? Запишите его название.")
print(u"Ответ: radiant_win (1, если победила команда Radiant, 0 — иначе)")

Вопрос: Какой столбец содержит целевую переменную? Запишите его название.
Ответ: radiant_win (1, если победила команда Radiant, 0 — иначе)


In [4]:
import time
import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.grid_search import GridSearchCV


def print_exec_time(func):
    def wrapper(*args, **kwargs):
        start_time = datetime.datetime.now()
        return_value = func(*args, **kwargs)
        end_time = datetime.datetime.now()
        print("Execution time of {} is {} (args: {}, kwargs: {})".format(
            func.__name__, end_time - start_time, args, kwargs)
        )
        
        return return_value

    return wrapper

# @print_exec_time
# def get_scores_by_n_estimators_grid(n_estimators):
    
#     grid = {'C': np.arange(0.1, 2.3, step=0.1)}
#     k_fold = KFold(len(y), n_folds=5, random_state=1, shuffle=True)
#     clf = GradientBoostingClassifier(
#         n_estimators=n_estimators,
#         random_state=1
#     )
#     gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=k_fold, n_jobs=-1)
#     gs.fit(X, y)


#     clf = GradientBoostingClassifier(
#         n_estimators=n_estimators,
#         random_state=1,
#         learning_rate=learning_rate,
#         max_depth=max_depth,
#         subsample=subsample
#     )

#     k_fold = KFold(len(y), n_folds=5, random_state=1, shuffle=True)
#     scores = cross_val_score(clf, X, y, cv=k_fold, scoring='roc_auc', n_jobs=-1)
#     return scores.mean()

@print_exec_time
def get_scores_by_n_estimators(n_estimators, learning_rate=0.1, max_depth=3, subsample=1.0):
    clf = GradientBoostingClassifier(
        n_estimators=n_estimators,
        random_state=1,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    )

    k_fold = KFold(len(y), n_folds=5, random_state=1, shuffle=True)
    scores = cross_val_score(clf, X, y, cv=k_fold, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

In [56]:
all_scores = list()
for n_estimators in [30]:
    all_scores.append((n_estimators, get_scores_by_n_estimators(n_estimators=n_estimators)))

Execution time of get_scores_by_n_estimators is 0:00:47.414812 (args: (), kwargs: {'n_estimators': 30})


In [71]:
all_scores

[(200, 0.71532383145178025),
 (400, 0.71471199024982923),
 (700, 0.70978495252079088),
 (1000, 0.70645424347752162)]

In [5]:
from sklearn.preprocessing import StandardScaler

X_scalled = StandardScaler().fit_transform(X)

In [9]:
import numpy as np
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(len(y), n_folds=5, shuffle=True, random_state=241)
clf = LogisticRegression(penalty='l2', random_state=241)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)
gs.fit(X_scalled, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [10]:
print("Best params is {}".format(gs.best_params_))
print("Best score is {}".format(gs.best_score_))

Best params is {'C': 0.01}
Best score is 0.716341465365


In [11]:
@print_exec_time
def get_scores_by_n_estimators_for_logit(X, C=0.1):
    clf = LogisticRegression(penalty='l2', random_state=241, C=C)

    k_fold = KFold(len(y), n_folds=5, random_state=241, shuffle=True)
    scores = cross_val_score(clf, X, y, cv=k_fold, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

In [13]:
print(u"Вопрос: Какое качество получилось у логистической регрессии над всеми исходными признаками?")
print(u"Ответ: {}".format(get_scores_by_n_estimators_for_logit(X=X_scalled, **gs.best_params_)))

Вопрос: Какое качество получилось у логистической регрессии над всеми исходными признаками?
Execution time of get_scores_by_n_estimators_for_logit is 0:00:06.123188 (args: (), kwargs: {'X': array([[-2.54436416,  1.54068827, -1.24422828, ..., -0.55115386,
         1.84600409, -1.12149424],
       [-2.54045236, -0.92779756, -0.29225805, ...,  0.67817009,
         0.43778816,  0.04394713],
       [-2.53923104,  1.54068827, -0.5686365 , ...,  0.67817009,
         0.43778816,  0.49028637],
       ..., 
       [ 1.09874571, -0.57515673,  1.42743012, ...,  0.67817009,
         0.43778816, -0.20401912],
       [ 1.09895204, -0.57515673,  1.48884755, ...,  0.67817009,
         0.43778816, -0.87352799],
       [ 1.1026479 ,  1.54068827, -0.04658831, ..., -0.55115386,
        -0.97042777, -0.79913812]]), 'C': 0.01})
Ответ: 0.716341465365


In [41]:
hero_columns = [
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero',
]

category_columns = hero_columns + ['lobby_type']

X_no_caterories = X.drop(category_columns, axis=1)

In [15]:
X_no_caterories_scalled = StandardScaler().fit_transform(X_no_caterories)

In [88]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(len(y), n_folds=5, shuffle=True, random_state=241)
clf = LogisticRegression(penalty='l2', random_state=241)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)
gs.fit(X_no_caterories_scalled, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [94]:
print("Best params is {}".format(gs.best_params_))
print("Best score is {}".format(gs.best_score_))

Best params is {'C': 0.01}
Best score is 0.716400950653


In [93]:
print(u"Как влияет на качество логистической регрессии удаление категориальных" 
      u"признаков (укажите новое значение метрики качества)?")
print(u"Ответ: {}".format(get_scores_by_n_estimators_for_logit(X=X_no_caterories_scalled, **gs.best_params_)))

Как влияет на качество логистической регрессии удаление категориальныхпризнаков (укажите новое значение метрики качества)?
Execution time of get_scores_by_n_estimators_for_logit is 0:00:15.227613 (args: (), kwargs: {'X': array([[-2.54436416,  1.40080818,  1.52597175, ..., -0.55115386,
         1.84600409, -1.12149424],
       [-2.54045236,  0.50131354, -0.08013929, ...,  0.67817009,
         0.43778816,  0.04394713],
       [-2.53923104,  0.50131354,  0.1510701 , ...,  0.67817009,
         0.43778816,  0.49028637],
       ..., 
       [ 1.09874571,  0.50131354,  0.29226667, ...,  0.67817009,
         0.43778816, -0.20401912],
       [ 1.09895204, -0.39818111, -0.17368203, ...,  0.67817009,
         0.43778816, -0.87352799],
       [ 1.1026479 , -0.39818111, -0.31840851, ..., -0.55115386,
        -0.97042777, -0.79913812]]), 'C': 0.01})
Ответ: 0.716400950653


In [112]:
all_heroes = np.unique(X[hero_columns])

In [16]:
# формирование мешка слов
X_with_bags = pandas.get_dummies(X, columns=hero_columns)

X_with_bags_scalled = StandardScaler().fit_transform(X_with_bags)

In [17]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(len(y), n_folds=5, shuffle=True, random_state=241)
clf = LogisticRegression(penalty='l2', random_state=241)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)
gs.fit(X_with_bags_scalled, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [18]:
print("Best params is {}".format(gs.best_params_))
print("Best score is {}".format(gs.best_score_))

Best params is {'C': 0.01}
Best score is 0.744640994532


In [20]:
get_scores_by_n_estimators_for_logit(X=X_with_bags_scalled, **gs.best_params_)

Execution time of get_scores_by_n_estimators_for_logit is 0:00:58.985496 (args: (), kwargs: {'X': array([[-2.54436416,  1.54068827,  1.40080818, ..., -0.0115638 ,
        -0.09736075, -0.20053848],
       [-2.54045236, -0.92779756,  0.50131354, ..., -0.0115638 ,
        -0.09736075, -0.20053848],
       [-2.53923104,  1.54068827,  0.50131354, ..., -0.0115638 ,
        -0.09736075, -0.20053848],
       ..., 
       [ 1.09874571, -0.57515673,  0.50131354, ..., -0.0115638 ,
        -0.09736075, -0.20053848],
       [ 1.09895204, -0.57515673, -0.39818111, ..., -0.0115638 ,
        -0.09736075, -0.20053848],
       [ 1.1026479 ,  1.54068827, -0.39818111, ..., -0.0115638 ,
        -0.09736075, -0.20053848]]), 'C': 0.01})


0.74464099453234023

In [21]:
all_heroes = np.unique(X[hero_columns])

In [36]:
X_pick = np.zeros((X.shape[0], max(all_heroes)))

for i, match_id in enumerate(X.index):
    for p in xrange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [46]:
from scipy.sparse import hstack

X_no_caterories_bag_words = hstack([X_no_caterories, X_pick]).toarray()

In [47]:
X_no_caterories_bag_words_scalled = StandardScaler().fit_transform(X_no_caterories_bag_words)

In [51]:
@print_exec_time
def get_grid_search_cv_logit(X):
    grid = {'C': np.power(10.0, np.arange(-5, 6))}
    cv = KFold(len(y), n_folds=5, shuffle=True, random_state=241)
    clf = LogisticRegression(penalty='l2', random_state=241)
    gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)
    gs.fit(X, y)
    return gs

In [52]:
gs = get_grid_search_cv_logit(X=X_no_caterories_bag_words_scalled)

Execution time of get_grid_search_cv_logit is 0:01:46.058793 (args: (), kwargs: {'X': array([[ -2.54436416e+00,   1.40080818e+00,   1.52597175e+00, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       [ -2.54045236e+00,   5.01313538e-01,  -8.01392943e-02, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       [ -2.53923104e+00,   5.01313538e-01,   1.51070097e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       ..., 
       [  1.09874571e+00,   5.01313538e-01,   2.92266672e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   1.64609695e+00],
       [  1.09895204e+00,  -3.98181106e-01,  -1.73682025e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       [  1.10264790e+00,  -3.98181106e-01,  -3.18408515e-01, ...,
         -4.83865990e-03,   0.00000000e+00,  -1.64440483e+00]])})


In [53]:
print("Best params is {}".format(gs.best_params_))
print("Best score is {}".format(gs.best_score_))

Best params is {'C': 0.01}
Best score is 0.751970490091


In [54]:
get_scores_by_n_estimators_for_logit(X=X_no_caterories_bag_words_scalled, **gs.best_params_)

Execution time of get_scores_by_n_estimators_for_logit is 0:00:11.045626 (args: (), kwargs: {'X': array([[ -2.54436416e+00,   1.40080818e+00,   1.52597175e+00, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       [ -2.54045236e+00,   5.01313538e-01,  -8.01392943e-02, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       [ -2.53923104e+00,   5.01313538e-01,   1.51070097e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       ..., 
       [  1.09874571e+00,   5.01313538e-01,   2.92266672e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   1.64609695e+00],
       [  1.09895204e+00,  -3.98181106e-01,  -1.73682025e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       [  1.10264790e+00,  -3.98181106e-01,  -3.18408515e-01, ...,
         -4.83865990e-03,   0.00000000e+00,  -1.64440483e+00]]), 'C': 0.01})


0.75197049009143802

In [69]:
features_test = pandas.read_csv('./features_test.csv', index_col='match_id')

X_test = features_test.fillna(0)

clf = LogisticRegression(penalty='l2', random_state=241, n_jobs=-1, **gs.best_params_)
clf.fit(X_no_caterories_bag_words_scalled, y)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [64]:
from scipy.sparse import hstack

def get_bag_of_words(X, all_heroes):
    X_pick = np.zeros((X.shape[0], max(all_heroes)))

    for i, match_id in enumerate(X.index):
        for p in xrange(5):
            X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
            
    return X_pick

X_pick = get_bag_of_words(X=X_test, all_heroes=all_heroes)

X_test_no_caterories = X_test.drop(category_columns, axis=1)
X_test_no_caterories_bag_words = hstack([X_test_no_caterories, X_pick]).toarray()
X_test_no_caterories_bag_words_scalled = StandardScaler().fit_transform(X_test_no_caterories_bag_words)

In [77]:
clf.predict_proba(X_no_caterories_bag_words_scalled)[2]

array([ 0.39716596,  0.60283404])

In [85]:
pred = clf.predict_proba(X_test_no_caterories_bag_words_scalled)[:, 1]

In [100]:
raw_data = {
    'match_id': X_test.index, 
    'radiant_win': pred
}

result_df = pandas.DataFrame.from_dict(raw_data)
result_df.set_index('match_id')

,radiant_win
match_id,
6,0.822689
7,0.756018
10,0.188658
13,0.856746
16,0.241456
18,0.380308
19,0.527367
24,0.562729
33,0.215821


In [103]:
result_df.to_csv('result.csv', index=False, columns=['match_id', 'radiant_win'])

In [105]:
print(u"Минимальное значение прогноза: {}".format(min(pred)))
print(u"Максимальное значение прогноза: {}".format(max(pred)))

Минимальное значение прогноза: 0.00870590076923
Максимальное значение прогноза: 0.996328715925
